# Use Case:  Language Translator
dataset consisting of pairs of English sentences and their French translation,

In [1]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np


In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'Datasets/csv_files/fra.txt'

In [3]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

In [4]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)


In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [6]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 90
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [7]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])


In [8]:
"""Turn the sentences into 3 Numpy arrays, encoder_input_data, decoder_input_data, decoder_target_data:

encoder_input_data is a 3D array of shape (num_pairs, max_english_sentence_length, num_english_characters) containing a one-hot vectorization of the English sentences
decoder_input_data is a 3D array of shape (num_pairs, max_french_sentence_length, num_french_characters) containing a one-hot vectorization of the French sentences
decoder_target_data is the same as decoder_input_data but offset by one timestep. decoder_target_data[:, t, :] will be the same as decoder_input_data[:, t + 1, :]
"""

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens),dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens),dtype='float32')


In [9]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [10]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [11]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [12]:
#Train a basic LSTM-based Seq2Seq model to predict decoder_target_data given encoder_input_data and decoder_input_data

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [13]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')


Epoch 1/100


125/125 [==============================] - 48s 280ms/step - loss: 1.3007 - accuracy: 0.7120 - val_loss: 1.2890 - val_accuracy: 0.6865
Epoch 2/100
125/125 [==============================] - 29s 232ms/step - loss: 1.0001 - accuracy: 0.7291 - val_loss: 1.0967 - val_accuracy: 0.6967
Epoch 3/100
125/125 [==============================] - 28s 228ms/step - loss: 0.9033 - accuracy: 0.7493 - val_loss: 1.4205 - val_accuracy: 0.6974
Epoch 4/100
125/125 [==============================] - 29s 232ms/step - loss: 0.8096 - accuracy: 0.7750 - val_loss: 0.8829 - val_accuracy: 0.7476
Epoch 5/100
125/125 [==============================] - 29s 234ms/step - loss: 0.7236 - accuracy: 0.7941 - val_loss: 0.8205 - val_accuracy: 0.7613
Epoch 6/100
125/125 [==============================] - 29s 234ms/step - loss: 0.6741 - accuracy: 0.8039 - val_loss: 0.7927 - val_accuracy: 0.7664
Epoch 7/100
125/125 [==============================] - 29s 232ms/step - loss: 0.6414 - accuracy: 0.8123 - val_loss: 0.758

c:\Users\AAHIL ALWANI\Desktop\FYP\Web App\Meme_AI\meme\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

In [15]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [16]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


In [20]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [22]:
# for seq_index in range(100):
#     # Take one sequence (part of the training set)
#     # for trying out decoding.
#     input_seq = encoder_input_data[seq_index: seq_index + 1]
#     #print(input_seq)
#     decoded_sentence = decode_sequence(input_seq)
#     print('-')
#     print('Input sentence:', input_texts[seq_index])
#     print('Decoded sentence:', decoded_sentence)

# Take input from user
input_sentence = input("Enter a sentence in english: ")

# Vectorize the input sentence
input_seq = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
for t, char in enumerate(input_sentence):
    input_seq[0, t, input_token_index[char]] = 1.
input_seq[0, t + 1:, input_token_index[' ']] = 1.

# Decode the input sentence
decoded_sentence = decode_sequence(input_seq)

# Print the decoded sentence
print('Decoded sentence:', decoded_sentence)

Enter a sentence in Roman Urdu:  Tom tightened the knot


IndexError: index 16 is out of bounds for axis 1 with size 16

In [ ]:
# Load the saved model
from keras.models import Model
loaded_model = load_model('s2s.h5')

# Define sampling models (these remain the same)
# ... (decoder models and helper functions)

# Function to pre-process user input
def pre_process_input(text):
    # Convert the text to a sequence of integers
    sequence = [input_token_index[char] for char in text]
    # Add padding to match the expected sequence length
    sequence = sequence + [input_token_index[' '] for _ in range(max_encoder_seq_length - len(sequence))]
    # Reshape the sequence to a 2D array
    sequence = np.array([sequence])
    # Create a placeholder for decoder input
    decoder_input = np.zeros((sequence.shape[0], max_decoder_seq_length, num_decoder_tokens), dtype='float32')
    # Set the start token for decoder input
    decoder_input[:, 0, target_token_index['\t']] = 1.
    # Reshape the encoder input to match the expected shape (batch_size, timesteps, input_dim)
    encoder_input = np.expand_dims(sequence, axis=-1)  # Add one more dimension
    return [encoder_input, decoder_input]

# Function to predict and decode the translated sentence
def predict_and_decode(text):
  # Preprocess the user input
  input_seq = pre_process_input(text)
  # Encode the input as state vectors
  states_value = loaded_model.predict(input_seq)

  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  # Populate the first character of target sequence with the start character.
  target_seq[0, 0, target_token_index['\t']] = 1.

  # Sampling loop for a single sequence
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict(
        [target_seq] + states_value)

    # Sample a token
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char

    # Exit condition: either hit max length or find stop character.
    if (sampled_char == '\n' or
       len(decoded_sentence) > max_decoder_seq_length):
      stop_condition = True

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.

    # Update states
    states_value = [h, c]

  return decoded_sentence

# Function to translate using the loaded model
def translate_sentence_loaded(input_text):
    input_seq = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    
    for t, char in enumerate(input_text):
        
        input_seq[0, t, input_token_index[char]] = 1.
    states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = list(target_token_index.keys())[list(target_token_index.values()).index(sampled_token_index)]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

# Get user input
user_input = input("Enter a sentence to translate: ")

# Translate the sentence
translated_sentence = predict_and_decode(user_input)

# Print the translated sentence
print("Translated sentence:", translated_sentence)

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense

# Sample dataset (replace with your own small dataset)
roman_urdu_sentences = [
    'mein ghar ja raha hoon \t',
    'kitab kahan hai \t',
    'tum kaise ho \t',
    'aaj mausam acha hai \t'
]
english_sentences = [
    'i am going home \t',
    'where is the book \t',
    'how are you \t',
    'the weather is nice today \t'
]

# Tokenization
input_characters = set(''.join(roman_urdu_sentences))
target_characters = set(''.join(english_sentences))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}



max_encoder_seq_length = max([len(sentence) for sentence in roman_urdu_sentences])
max_decoder_seq_length = max([len(sentence) for sentence in english_sentences])

encoder_input_data = np.zeros((len(roman_urdu_sentences), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(roman_urdu_sentences), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(roman_urdu_sentences), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (roman_urdu_sentence, english_sentence) in enumerate(zip(roman_urdu_sentences, english_sentences)):
    for t, char in enumerate(roman_urdu_sentence):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(english_sentence):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

# Define the model
latent_dim = 256

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Train the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=1, epochs=50)

# Save the model
model.save('roman_model.h5')

# Inference
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)



In [ ]:
# Load the saved model
loaded_model = load_model('roman_model.h5')

# Function to translate using the loaded model
def translate_sentence_loaded(input_text):
    input_seq = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    
    for t, char in enumerate(input_text):
        
        input_seq[0, t, input_token_index[char]] = 1.
    states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = list(target_token_index.keys())[list(target_token_index.values()).index(sampled_token_index)]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

# Test the loaded model
roman_urdu_input = 'tum \t'
english_translation_loaded = translate_sentence_loaded(roman_urdu_input)
print(f'Roman Urdu: {roman_urdu_input}')
print(f'English Translation: {english_translation_loaded}')